In [1]:
import torch
import numpy as np
from tqdm import tqdm
import torch.optim as optim
import copy

## 函数实现

In [2]:
def step1(x,y,lam,epoch=5000):
    
 
    n1,n2 = y.shape
    m1,m2 = x.shape
    
    T = torch.tensor(np.where(y==0,0,1))#
    gama = torch.randn(m2,n2,requires_grad=True,dtype=torch.float)
    z = torch.randn(n1,n2,requires_grad=True,dtype=torch.float)
    
    optimizer = optim.Adam([gama,z], lr=0.1)
    for epoch in tqdm(range(1000)):
        optimizer.zero_grad()  # 清空之前的梯度
        a1 = torch.mul(T,torch.log(torch.sigmoid(x@gama)+z))
        a2 = torch.mul((1-T),torch.log(1-(torch.sigmoid(x@gama)+z)))
        a3 = lam*torch.linalg.norm(z, ord='nuc')
        loss = -(a1+a2-a3)
        loss.backward()
        # 参数更新
        optimizer.step()
    
   
    
    
    thta = torch.sigmoid(x@gama)+z
    
    
    return thta.detach()


    
    
    



def step2(sita,y,lam=0.01,num_epochs=1000):
    sita = sita**(-0.5)
    T = torch.tensor(np.where(y==0,0,1))
    # print(T)
    n1,n2 = y.shape
    A = torch.randn((n1,n2), requires_grad=True,dtype=torch.float)
    optimizer = optim.Adam([A], lr=0.1)
    for epoch in tqdm(range(num_epochs)):
        optimizer.zero_grad()  # 清空之前的梯度
        w = torch.mul(T,sita)
        loss = torch.norm(torch.mul(w,(A-y)), p=2)/(n1*n2)+lam*torch.linalg.norm(A, ord='nuc')
        loss.backward()
        # 参数更新
        optimizer.step()
        # if (epoch+1) % (num_epochs/5) == 0:
        #     print(f'Epoch {epoch+1}: Loss = {loss.item():.4f}')
    return A

def compute_missing_probabilities(matrix):
    # 获取矩阵的大小
    m, n = matrix.shape

    # 创建一个与原始矩阵相同大小的矩阵，用于存储每个元素的缺失概率
    missing_probs = np.zeros((m, n))

    # 计算行缺失概率
    row_probs = 1 - np.count_nonzero(matrix, axis=1) / n

    # 计算列缺失概率
    col_probs = 1 - np.count_nonzero(matrix, axis=0) / m

    # 计算每个元素的缺失概率
    for i in range(m):
        for j in range(n):
            missing_probs[i, j] = row_probs[i] * col_probs[j]

    return missing_probs


## 参数设置

In [99]:
torch.manual_seed(140)
np.random.seed(110)
n1,n2 = 100,100
nums_v = 20


## main

In [118]:
seed = 12
torch.manual_seed(seed)
np.random.seed(seed)
A = torch.rand(n1,n2)*10



y = copy.deepcopy(A)+torch.randn(n1,n2)*0.5

for i in range(n1):
    for j in range(n2):
        
        p = np.random.uniform(0,1,1)
        
        if p>=0.5:
            y[i,j] = 0


thta = step1(w=0,y=y,nums_v=nums_v,lam=1)
x = step2(thta,y,lam=0.000001)

100%|██████████| 1000/1000 [00:01<00:00, 733.00it/s]


In [97]:
thta

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [30]:
y

tensor([[0.7576, 0.0000, 0.4031,  ..., 0.0000, 0.2873, 0.0000],
        [0.9579, 0.0000, 0.0000,  ..., 0.8212, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.5069, 0.0000, 0.2210],
        ...,
        [0.3813, 0.5065, 0.0000,  ..., 0.0000, 0.6924, 0.2933],
        [0.0000, 0.0844, 0.0000,  ..., 0.6694, 0.0000, 0.5073],
        [0.7087, 0.0000, 0.0000,  ..., 0.4243, 0.0811, 0.0000]])

In [119]:
T = torch.tensor(np.where(y==0,0,1))
torch.linalg.norm(torch.mul(x,T)-torch.mul(A,T),ord=2)

tensor(6.7926, grad_fn=<AmaxBackward0>)

In [120]:
# thta = torch.ones(y.shape)
# thta = compute_missing_probabilities(y)
# thta = torch.tensor(thta)
thta = torch.ones(y.shape)*0.5
x = step2(thta,y,lam=0.000001)
torch.linalg.norm(torch.mul(x,T)-torch.mul(A,T),ord=2)

100%|██████████| 1000/1000 [00:01<00:00, 806.63it/s]


tensor(6.7879, grad_fn=<AmaxBackward0>)